# The Cournot Model

This project provides an examination of the dynamics of the Cournot model, exploring how firms interact to maximize profits while considering the output levels of their competitors. Initially assuming two firms in the market, we begin by constructing the model, detailing the mathematical basis, which includes profit functions, best response functions, and the iterative process of equilibrium determination. Following that, we extend our analysis to involve varying parameters and numbers of firms, to provide clearer insights into understanding the markets, their equilibrium conditions, and the competitive dynamics at play as the number of firms varies. Through this project, we aim to offer intuitive insights into the behavior of firms and the implications for market outcomes.

Imports and set magics:

In [ ]:
import sympy as sm
import numpy as np
import warnings 
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, Layout
from IPython.display import Markdown

warnings.filterwarnings("ignore", category=RuntimeWarning)

# 1. Model description

**1.1 Cournot Model with 2 Firms** 

Under Cournot competition, firms determine their output quantities, after which the market price is established. The market equilibrium is a Nash equilibrium, where neither firm has an incentive to unilaterally deviate from its chosen output level. We consider an economy where two identical firms produce the same homogenous good.

The Cournot model makes the following assumptions:

* The firms independently and simultaneously choose the quantity $q_i$ they wish to produce, denoted as $q_1$ and $q_2$ respectively.

* Each firm aims to maximize its own profit, given the quantity produced by the other firm.

* Each firm has the same constant marginal cost of production, c.

* The market price is determined by the total supply and a known demand function.

* There is no cooperation between the firms.

Each firm $i$ aims to maximize its profit by choosing its production level, taking the output of the other firm $j$ as fixed. The profit function for firm $i$ is given by

$$
\begin{aligned}
\Pi_i (q_{-i}, q_i) &= p(q)q_i - c_iq_i\\
&= (p(q_{-i} + q_i) - c_i)q_i
\end{aligned}
$$


We can rewrite the profit function to be a function of the quantity. Thus, the profit functions for the two firms can be written as

$$
\begin{aligned}
\Pi_i (q_{-i}, q_i) = (a - q_{-i} - q_i - c)q_i\\
\end{aligned}
$$

The method to solve it can be described as follows:

1. Write up the profit functions and take the derivative with respect to one's quantity.

2. From the first order conditions (FOCs), find the Best-Response (BR) functions given the quantity chosen by the other player.

3. Substitute in the BR quantity for the other player to solve for the equilibrium quantities.

## 2. Analytical Solution

We start by solving the model analytically. 

First, we write up the parameters and profit functions.

In [ ]:
# Declare the symbols
a, c, q1, q2 = sm.symbols('a c q_1 q_2')
Q = q1 + q2

# Profit functions for firm 1 and firm 2
profit1 = q1*(a - Q - c)
profit2 = q2*(a - Q - c)

# Print the profit functions
display(sm.Eq(sm.symbols('profit1'), profit1))
display(sm.Eq(sm.symbols('profit2'), profit2))

Second, we derive the first order conditions (FOC) of the profit function given the production for the firm. 

In [ ]:
# Take the FOC of each firm's profit function
foc1 = sm.diff(profit1, q1)
foc2 = sm.diff(profit2, q2)

# Print the profit functions
display(sm.Eq(sm.symbols('FOC1'), foc1))
display(sm.Eq(sm.symbols('FOC2'), foc2))

Then we set FOCs equal to 0 and solve for the quantity of the firm to get the firm's Best Response (BR) function. 

In [ ]:
# Solve these to find each firm's best response function
sol1 = sm.solve(sm.Eq(foc1, 0), q1)
sol2 = sm.solve(sm.Eq(foc2, 0), q2)

# Print the profit functions
display(sm.Eq(sm.symbols('q1'), sol1[0].together() ))
display(sm.Eq(sm.symbols('q2'), sol2[0].together() ))

This is the firm’s BR function given the quantity chosen by the other firm.

To find the equilbrium quantity $(q_1^*, q_2^*)$ we solve the BR functions iteratively. The Cournot equilibrium for firm 1, $q_1^*$ is a solution of firm 1's BR function when $q_2 = q_2^*$, and $q_2^*$ is a solution of firm 2's reaction function when $q_1 = q_1^*$.


In [ ]:
# Solving the equilibrium
cournot_eqm = sm.solve((sol1[0] - q1, sol2[0] - q2), (q1, q2))

# Define new symbols for the solutions
q_i = sm.symbols('q_i^*')

# Print the solutions nicely
display(sm.Eq(q_i, cournot_eqm[q1].together()))

## 3.Numerical Solution

In the next section, we'll be solving the model numerically, maintaining the assumption of two firms competing in the market. By solving the model numerically, we can gain a more nuanced understanding of how the firms' output decisions and market outcomes are shaped by the underlying parameters of the model. 

**3.1 Defining the model**

In the upcoming code section, we'll be using a Python package we've developed, known as modelproject. This package houses our Cournot competition model, referred to as Cournotmodel.

The Cournot model implements the Cournot competition model for a duopoly. It provides methods for determining each firm's best response strategy, solving for market equilibrium, calculating production levels, and visualizing the data.
We set the parameters equal

$$
\begin{aligned}
a &= 11\\
b &=  1\\
c &=  2
\end{aligned}
$$

The parameter 'a' is the market demand intercept, 'b' is the market demand slope and 'c' is the cost per unit produced. 

In [ ]:
# Importing the Cournotmodel class from modelproject module
from modelproject import Cournotmodel

# Set the parameters
a = 11  
b = 1   
c = 2  

# Create a Cournotmodel with given parameters a, b, and c
model = Cournotmodel(a, b, c)  

**3.2 Equilibrium, BR functions and Profit Function**

Next we find the Cournot/Nash equilibrium quantities, price, and profit for a Cournot competition model using an initial guess of quantities for two firms.

In [ ]:
# Set up the initial guess for the quantities produced by firm 1 and firm 2
initial_guess = [1, 1]  

# Call the 'find_equilibrium' method of the  'Cournotmodel' using the initial guess as an argument.
q_star, p_star, pi_star = model.find_equilibrium(initial_guess)

# Prepare a multi-line string with the equilibrium information. It includes Nash Equilibrium quantities, 
# price at the Nash Equilibrium and profit at the Nash Equilibrium.
equilibrium_info = f"""
The Cournot/Nash Equilibrium is:
    $q_1^* = q_2^*$ = {q_star:.2f}

The price at the Nash equilibrium is:
    $p^*$ = {p_star:.2f}

The profit at the Nash equilibrium is:
    $\Pi$ = {pi_star:.2f}
"""

# Display the equilibrium information 
display(Markdown(equilibrium_info))

In a duopoly market, we find that the equilibrium quantities amount to 3 units per firm, with the market price standing at 5. This implies that the two firms produce the same quantity of a good as they both operate under the same profit function. Each firm is expected to achieve an equal profit, which in this case, amounts to 3 units.

We use the best response functions for firms 1 and 2 to plot the intersection, representing the Cournot Equilibrium. The best response function is determined by the quantity produced by the other firm. Therefore, we first identify firm 2's best response function, then incorporate it into firm 1's response function to determine the equilibrium quantities.

We do so by first creating an array of numbers from 0 to 10 with an interval of 0.1, which represents different production levels for firm 1. Then, using these production levels, it calculates the production levels for both firm 1 and firm 2 at these points, taking firm reactions into consideration. Finally, it uses these calculated production levels to create a scatter plot visualizing the relationship between the production levels of the two firms.

In [ ]:
# Create an array representing possible production levels for Firm 1
production_level_f1 = np.arange(0, 10, 0.1)

# Calculate production levels for both Firm 1 and Firm 2 at these points,
# taking into account the reaction functions of the firms.
production_level_f1_2, production_level_f2 = model.calculate_production_levels(production_level_f1)

# Use the calculated production levels to create a plot of the relationship between the production levels of the two firms.
model.plot_scatter(production_level_f1_2, production_level_f2)

From the graph we see that the intersection of the BR functions is at (3, 3), which is the equilibrium quantities. This means that the two firms will produce 3 items each. The BR functions are crossing the other firms output axis equal 4.5. It means that the firm will produce 4.5 items when the other firm is not producing any. This impliesd that the total output of the market is smaller for a monopolist than duopoly market. 

Next we want to plot the profit function. As the firms are identical we only do it for firm 2. The used method provides a visual representation of how the firm's profit changes with different production quantities. 

In [ ]:
# Generates a 3D plot of the profit functions of firm 2 given their production quantities.
model.plot_profit_functions()

The plot above visualizes the profit function for firm 2. Given the identical firms in this model, the profit function for firm 1 would also mirror this same plot. As depicted, when firm 1's production level is zero, firm 2's profit equals zero as well. Conversely, when firm 1 is not producing, firm 2 ramps up production to 4.5 units, achieving a profit around the 20.

The red dot symbolizes the Nash Equilibrium point, where both firms reach their optimal production quantities. This visualization therefore provides a significant perspective into the dynamics of strategic interactions and decision-making within the scope of the Cournot competition model.

# 4. Different parameter values

In this section, we explore the impact of different parameter values on the Cournot model and its equilibrium. By varying the values of the parameters a, b, and c, we can observe how these changes affect the market and the equilibrium quantities produced by the firms. This enables us to assess the sensitivity of the model to parameter variations.

The following code provides an interactive visualization of the Cournot model, allowing users to explore how different parameter values affect the demand curve and equilibrium point. First we define and calculate the equilibrium price and quantity. The update_plot function is used to interact with the cournot_model function. It takes the slider values for a, b, and c as input and updates the plot accordingly. The interact function creates sliders for the parameters and connects them to the update_plot function, allowing the user to dynamically adjust the parameter values and see the corresponding changes in the plot.

In [ ]:
# Calculate the equilibrium quantity and price
def cournot(N, a, b, c):
    q = (a - c)/(N + 1) * b  
    p = a - N * q  
    return p, q

# Set up the model with number of firms, the equilibrium price and quantity and demand curve
def cournot_model(a, b, c):
    N = 2  
    p, q = cournot(N, a, b, c)  

    x = np.linspace(0, a, 100) 
    demand = a - N * x 

    # Create a Plotly figure with subplots
    fig = go.Figure()  
    fig.add_trace(go.Scatter(x=x, y=demand, name='Demand'))  
    fig.add_trace(go.Scatter(x=[q], y=[p], mode='markers', name='Equilibrium'))  

    # Set the layout for the figure - title and labels, hover and axes
    fig.update_layout(
        title=f'{N} firms, a={a}, b={b}, c={c}',  
        xaxis_title='Quantity',  
        yaxis_title='Price',  
        hovermode='closest'  
    )

    # Format hover label values to two decimal places
    fig.update_traces(hovertemplate='Price: %{y:.2f}<br>Quantity: %{x:.2f}')

    return fig

# Create sliders for a, b, and c
a_slider = FloatSlider(min=0, max=20, step=1, value=11, description='a:', layout=Layout(width='50%'))
b_slider = FloatSlider(min=0, max=5, step=0.1, value=1, description='b:', layout=Layout(width='50%'))
c_slider = FloatSlider(min=0, max=5, step=0.1, value=2, description='c:', layout=Layout(width='50%'))

# Use the sliders to interact with the cournot_model function and update the plot based on the slider values
def update_plot(a, b, c):
    fig = cournot_model(a, b, c)  
    fig.show()

interact(update_plot, a=a_slider, b=b_slider, c=c_slider)


The parameter a is the demand intercept in the market. So a high value a implies a high demand for the good in the market. 

The parameter b is the slope of the demand. For b going toward infinity, the dependence on the other firm's price on one's own production increases. A high value of b therefore means a the demand curve becomes steeper, and a larger decrease in the market price occurs for each unit increase in the quantity produced. This has the effect of reducing the quantity of the good that each firm produces in equilibrium. 

The parameter c is the cost of production for the firm. It is the cost for the firm to produce one more unit. If c increases, the marginal cost of production for the firm increases, which makes it more expensive for firms to produce each unit of output. 

Overall, we see that varying the parameter values has an impact on the resulting equilibrium quantities produced by the firms.

# 5. Different number of firms in the market

To analyze the impact of having more firms in the market, we can modify the code to consider a variable number of firms instead of just two. By increasing the number of firms, we can observe how competition and market outcomes change.

We uses the class 'Firms'. It creates an instance of the Firms class with parameter baseline values a=11, b=1, and c=2. Finally, it calls the plot_firm_convergence() method of the Firms instance to generate a plot that illustrates the convergence of profits and prices as the number of firms in the market increases.

In [ ]:
from modelproject import Firms

# Instantiate the model with a = 11, b = 1 and c = 2
model = Firms(a=11, b=1, c=2)

# Call the new plot function
model.plot_firm_convergence()

When more firms are present in the market, the dynamics of competition change. As the number of firms increases, the competition intensifies, leading to potential shifts in equilibrium quantities, prices, and profits.

If the number of firms would increase further towards infinity, the equilibrium price will go towards the cost

$$
\begin{aligned}
p^* \rightarrow c
\end{aligned}
$$

and the profit will go towards 0

$$
\begin{aligned}
\pi \rightarrow 0
\end{aligned}
$$

This corresponds to the results that we would get under perfect competition. The price will be pushed down to the marginal cost, the quantity will increase, the profit will go towards 0 and consumer surplus will increase. Exactly as under perfect competition. 

# 6. Conclusion

We have looked a Cournot model with different parameter values and different number of firms in the market.

When having two firms in the market we got a equilibrium price equal 5.00 and equilbrium quantity equal 3.00. When increasing tbe number of firms in the market the equilibrium price decreased but the total quantity of the market increased. 

By increasing the number of firms in the market further, the price will go towards the marginal cost, the quantity will increase and the profit will go towards 0. 